In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('final_merged_data.csv')

In [3]:
df['item_name'].unique()

array(['10 pc Grilled Wings Combo', '8 pc Grilled Wings Combo',
       '8 pc Spicy Wings Combo', 'Ranch Dip - Regular',
       '50 pc Grilled Wings', 'Regular Buffalo Fries',
       '20pc Spicy Feast Deal', '20 pc Grilled Wings',
       '6 pc Grilled Wings Combo', '10 pc Grilled Wings',
       'Fried Corn - Regular', 'Ranch Dip - Large', 'Veggie Sticks Spicy',
       'Flavor Platter', 'Large Fruit Punch',
       'Honey Mustard Dip - Regular', 'Voodoo Fries - Regular',
       '10 pc Spicy Wings', '2 pc Crispy Strips', 'Add 5 Grilled Wings',
       '24 pc Family Grilled Wings', '6 pc Spicy Wings Combo',
       'Chicken Sub Combo', '15 pc Spicy Wings',
       '10 pc Spicy Wings Combo', 'Add 5 Spicy Wings',
       'Blue Cheese Dip - Regular', 'Chicken Sub',
       '30 pc Family Grilled Wings', '10 pc Mixed Wings Combo',
       'Dipping Sauce', 'Large Buffalo Fries', '6 pc Mixed Wings Combo',
       '40 pc Family Grilled Wings', '32 Oz Soda',
       '100 pc Family Grilled Wings', '15 pc Gri

In [5]:
grouped_df = df.groupby("ORDER_ID").agg({
    "CUSTOMER_ID": "first",# Single value per order
    "CUSTOMER_TYPE":"first",
    "STORE_NUMBER":"first",
    "ORDER_CREATED_DATE": "first",
    "ORDER_CHANNEL_NAME": "first",
    "ORDER_OCCASION_NAME": "first",
    "CITY": "first",
    "STATE": "first",
    "POSTAL_CODE": "first",
    "item_name": list,               # Convert items to list
    "item_total": "sum"              # Total order amount
}).reset_index()

In [6]:
grouped_df.rename(columns={"item_total": "total_order_amount"}, inplace=True)

In [7]:
grouped_df["total_items"] = grouped_df["item_name"].apply(len)

In [8]:
print(grouped_df.head())

   ORDER_ID  CUSTOMER_ID CUSTOMER_TYPE  STORE_NUMBER ORDER_CREATED_DATE  \
0     19042    948014694    Registered          2156         2024-12-19   
1     21421    681204572    Registered          1913         2025-03-15   
2     49314    705735618    Registered          4750         2024-04-08   
3     58820    914568760         Guest          1161         2024-12-31   
4     62930    304069593    Registered          4541         2024-05-04   

  ORDER_CHANNEL_NAME ORDER_OCCASION_NAME       CITY    STATE POSTAL_CODE  \
0            Digital                ToGo  GRAPEVINE       TX       76051   
1            Digital                ToGo     IRVING       TX       75063   
2            Digital                ToGo    Unknown  Unknown     Unknown   
3            Digital                ToGo    Unknown  Unknown  78723-2429   
4            Digital                ToGo     MACOMB       MI       48044   

                                           item_name  total_order_amount  \
0               

In [9]:
grouped_df.shape

(1414410, 13)

In [10]:
grouped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1414410 entries, 0 to 1414409
Data columns (total 13 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   ORDER_ID             1414410 non-null  int64  
 1   CUSTOMER_ID          1414410 non-null  int64  
 2   CUSTOMER_TYPE        1414410 non-null  object 
 3   STORE_NUMBER         1414410 non-null  int64  
 4   ORDER_CREATED_DATE   1414410 non-null  object 
 5   ORDER_CHANNEL_NAME   1414410 non-null  object 
 6   ORDER_OCCASION_NAME  1414410 non-null  object 
 7   CITY                 1414410 non-null  object 
 8   STATE                1414410 non-null  object 
 9   POSTAL_CODE          1414410 non-null  object 
 10  item_name            1414410 non-null  object 
 11  total_order_amount   1414410 non-null  float64
 12  total_items          1414410 non-null  int64  
dtypes: float64(1), int64(4), object(8)
memory usage: 140.3+ MB


In [11]:
grouped_df.to_csv("orders_grouped_cleaned(1).csv", index=False)

In [12]:
# Step 8: Recommend items for a given cart
def recommend_items_for_cart(cart_items, k=5, top_n=3):
    cart_string = ' '.join(i.replace(" ", "_") for i in cart_items)
    cart_vec = vectorizer.transform([cart_string])
    distances, indices = knn.kneighbors(cart_vec, n_neighbors=k)

    recommended = []
    for idx in indices[0]:
        neighbor_items = order_df.iloc[idx]['parsed_items']
        recommended.extend(neighbor_items)

    # Filter out items already in the cart and get top-N
    filtered = [item for item in recommended if item not in cart_items]
    return pd.Series(filtered).value_counts().head(top_n).index.tolist()